In [ ]:
from google.colab import files
import zipfile
import os
import pandas as pd
import glob

uploaded = files.upload()


In [ ]:
zip_path = next(iter(uploaded))
extract_folder = "/content/csvs"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)


In [ ]:
archivos = glob.glob(f"{extract_folder}/**/*.csv", recursive=True)

dataframes_validos = []
columnas_clave = ['Producto', 'Descripción de careacterística', 'Cant. movida', 'Hueco']

for archivo in archivos:
    try:
        df = pd.read_csv(archivo, sep=";", encoding="latin1")
        if all(col in df.columns for col in columnas_clave):
            dataframes_validos.append(df)
            print(f"✅ {os.path.basename(archivo)} cargado")
        else:
            print(f" Columnas faltantes en {os.path.basename(archivo)}")
    except Exception as e:
        print(f" Error leyendo {os.path.basename(archivo)}: {e}")

if not dataframes_validos:
    raise ValueError("No se cargaron archivos válidos")

df_total = pd.concat(dataframes_validos, ignore_index=True)


In [ ]:
df_total = df_total.rename(columns={
    'Producto': 'producto',
    'Descripción de careacterística': 'talla',
    'Cant. movida': 'cantidad',
    'Hueco': 'ubicacion'
})

for col in ['producto', 'talla', 'cantidad', 'ubicacion']:
    if col not in df_total.columns:
        raise ValueError(f" Falta la columna requerida: '{col}'")

df_total['producto'] = df_total['producto'].astype(str).str.upper().str.strip()
df_total['talla'] = df_total['talla'].astype(str).str.upper().str.strip()
df_total['ubicacion'] = df_total['ubicacion'].astype(str).str.upper().str.strip()
df_total['cantidad'] = pd.to_numeric(df_total['cantidad'], errors='coerce').fillna(0)

df_total['ubicacion'] = df_total['ubicacion'].replace({
    'LOGINSER CENTRAL': 'LOGINSER',
    'LOGINSER': 'LOGINSER',
    'TIENDA': 'TIENDA',
    'TRANSITO LOGINSER': 'LOGINSER'
})

df_total = df_total[df_total['ubicacion'].isin(['TIENDA', 'LOGINSER'])]

df_total.head()

In [ ]:
from IPython.display import display

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

display(df_total)


In [ ]:
df_agrupado = df_total.groupby(['producto', 'talla', 'ubicacion'])['cantidad'].sum().reset_index()
display(df_agrupado)

In [ ]:
%pip install fpdf2 tabulate

In [ ]:

excel_output_path = "/content/df_agrupado.xlsx"
df_agrupado.to_excel(excel_output_path, index=False)

print(f"Excel file saved to: {excel_output_path}")

Excel file saved to: /content/df_agrupado.xlsx


In [ ]:
from google.colab import files

excel_output_path = "/content/df_agrupado.xlsx"
try:
  files.download(excel_output_path)
except FileNotFoundError:
  print(f"Error: The file {excel_output_path} was not found. Please ensure the Excel file was generated successfully.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from fpdf import FPDF
from google.colab import files

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 11)
        self.cell(0, 10, 'Resumen agrupado de albaranes', ln=True, align='C')
        self.ln(4)

        self.set_font("Arial", 'B', 9)
        headers = ['producto', 'talla', 'ubicacion', 'cantidad']
        for i, header in enumerate(headers):
            self.cell(col_widths[i], 8, header.upper(), border=1, align='C')
        self.ln()

col_widths = [180, 40, 35, 22]
line_height = 8

pdf = PDF(orientation='L', unit='mm', format='A4')
pdf.add_page()

pdf.set_font("Arial", '', 8)

for _, row in df_agrupado.iterrows():
    producto_lines = len(pdf.multi_cell(col_widths[0], line_height, str(row['producto']), border=0, split_only=True))
    row_height = producto_lines * line_height

    if pdf.get_y() + row_height > 190:
        pdf.add_page()

    y_before = pdf.get_y()
    x = pdf.get_x()

    pdf.multi_cell(col_widths[0], line_height, str(row['producto']), border=1)
    y_after = pdf.get_y()
    actual_row_height = y_after - y_before

    pdf.set_xy(x + col_widths[0], y_before)
    pdf.multi_cell(col_widths[1], actual_row_height, str(row['talla']), border=1, align='C')

    pdf.set_xy(x + col_widths[0] + col_widths[1], y_before)
    pdf.multi_cell(col_widths[2], actual_row_height, str(row['ubicacion']), border=1, align='C')

    pdf.set_xy(x + col_widths[0] + col_widths[1] + col_widths[2], y_before)
    pdf.multi_cell(col_widths[3], actual_row_height, str(int(row['cantidad'])), border=1, align='C')

    pdf.set_y(y_before + actual_row_height)

pdf_output_path = "/content/df_agrupado.pdf"
pdf.output(pdf_output_path)

files.download(pdf_output_path)
